In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, Layout, ButtonStyle
from scipy.integrate import odeint
import math
import pandas as pd
import csv
from IPython.display import display, Javascript

In [ ]:
t = np.linspace(0, 1000000, 1000)
rho_f = 1000  # Medium Density (Kg/m^3)

# Define global variables
C_l_data = None

# Define the KLa empirical equation as a function
def dcdt(C_l, t, T, Np, N, Di, v_s, rho_f, H, D):
    dCdt = (1.022 ** (T - 20)) * 0.026 * (
                (Np * 0.5 * rho_f * N ** 3 * Di ** 5 / (math.pi * D ** 2 * H)) ** 0.4 * v_s ** 0.5 * (100 - C_l))
    return dCdt


# Function to update CSV file
def update_csv_file(_):
    global C_l_data
    if C_l_data is not None:
        with open('KLA_resources/oxygen_concentration_data.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Time (s)', 'Oxygen concentration (%)'])
            for time, concentration in zip(t, C_l_data.flatten()):
                writer.writerow([time, concentration])
        print("CSV file updated successfully!")
    else:
        print("Please calculate concentration first.")

# Function to plot the oxygen concentration versus time
def plot_C(T, Np, N, Di, D, H, C_l0):
    global C_l_data
    # Calculate oxygen concentration over time
    C_l_data = odeint(dcdt, C_l0, t, args=(T, Np, N, Di, D, H, v_s, rho_f))
    
    # Sort the time array
    sorted_t = np.sort(t)
    # Interpolate the concentration data to match the sorted time array
    interpolated_C_l = np.interp(sorted_t, t, C_l_data.flatten())
    
    # Plotting with Matplotlib
    plt.figure(figsize=(10, 6))
    plt.plot(sorted_t, interpolated_C_l, label='Oxygen concentration (%)')
    plt.title('Oxygen concentration vs. time')
    plt.xlabel('Time in s')
    plt.ylabel('Oxygen concentration in %')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Export the data to CSV file
    update_csv_file(None)

    
# Define style for slider descriptions
style = {'description_width': 'initial'}

# Define sliders for input parameters with customized style
T_slider = FloatSlider(min=20, max=50, step=1, value=30, description='Temperature (°C) \u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0', layout=Layout(width='800px', height='auto'), style=style)
Np_slider = FloatSlider(min=0, max=36, step=6, value=6, description='Impeller Power Number (Np) \u00A0\u00A0\u00A0', layout=Layout(width='800px', height='auto'), style=style)
D_slider = FloatSlider(min=0.05, max=0.15, step=0.01, value=0.1, description='Reactor inner Diameter (m) \u00A0\u00A0\u00A0\u00A0\u00A0\u00A0', format='.5f', layout=Layout(width='800px', height='auto'), style=style)
H_slider = FloatSlider(min=0.1, max=1.0, step=0.01, value=0.5, description='Reactor height (m) \u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0', format='.5f', layout=Layout(width='800px', height='auto'), style=style)
N_slider = FloatSlider(min=50, max=2000, step=10, value=100, description='Rotation Speed (s^-1) \u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0',  layout=Layout(width='800px', height='auto'), style=style)
Di_slider = FloatSlider(min=0.01, max=0.05, step=0.005, value=0.015, description='Impeller Diameter (m) \u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0\u00A0', layout=Layout(width='800px', height='auto'), style=style)
C_l0_slider = FloatSlider(min=0, max=50, step=1, value=1, description='Initial oxygen concentration (%)', layout=Layout(width='800px', height='auto'), style=style)

v_s = float(input("Enter Aeration Rate (vvs): "))

# Button style
button_style = ButtonStyle(
    button_color='#FF4642',  
    font_weight='bold',
    text_color='white') 

button_layout = Layout(width='226px',    # Assuming this is your preferred button width
                       height='40px',    # Assuming this is your preferred button height
                       border='1px solid #1F6F44',  # Assuming this is your preferred border style
                       margin='10px',
                       padding='5px',
                       border_radius='20px')  # Assuming this is your preferred border radius

# Button to update CSV file
update_button = Button(description="Save Data", style=button_style, layout=button_layout,)
update_button.on_click(update_csv_file)

# Use interact to create interactive sliders
interact(plot_C, T=T_slider, Np=Np_slider, N=N_slider, D=D_slider, Di=Di_slider,H=H_slider, C_l0=C_l0_slider);

# Display the button
display(update_button)


Enter Aeration Rate (vvs):  0.1


In [ ]:
def convert_csv_to_excel():
    # Read CSV file
    df = pd.read_csv('KLA_resources/oxygen_concentration_data.csv')

    # Convert to Excel and overwrite existing file
    excel_file_path = 'KLA_resources/kLa_experiment_data.xlsx'
    df.to_excel(excel_file_path, index=False)

    print("CSV file has been converted to Excel.")

    # Optionally, prompt download in Jupyter Notebook
    display(Javascript(f"""
        var link = document.createElement('a');
        link.href = 'files/{excel_file_path}';
        link.download = '{excel_file_path}';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
    """))

# Custom Button Style to match your other buttons
button_style = ButtonStyle(button_color='#1F6F44',  # Assuming this is your preferred button color
                           font_weight='bold',
                           text_color='white')

# Custom Button Layout to match your other buttons
button_layout = Layout(width='226px',    # Assuming this is your preferred button width
                       height='40px',    # Assuming this is your preferred button height
                       border='1px solid #1F6F44',  # Assuming this is your preferred border style
                       margin='10px',
                       padding='5px',
                       border_radius='20px')  # Assuming this is your preferred border radius

convert_button = Button(description="Save to Excel",
                        layout=button_layout,
                        style=button_style)

convert_button.on_click(lambda b: convert_csv_to_excel())

# Display the button
display(convert_button)
